#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: []
- type: ap
- week: 26

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

# makes the sqlite.db file


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User
    - db.Model inheritance
    - _init_ method
    - ```@property```, ```@<column>.setter```
    - create, read, update, delete methods

In [4]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class Player(db.Model): # way to inherit the propeties into the user from the template, let the template do database stuff
    # template for the properties we want the user to have
    __tablename__ = 'football'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _number = db.Column(db.Integer, unique=False, nullable=False)
    _wins = db.Column(db.Integer, unique=False, nullable=False)
    _losses = db.Column(db.Integer, unique=False, nullable=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, number, wins, losses): # initialiazes the attributes within the class, uses parameters to be recieved in the init method to instantiate the object of the template user
        self._name = name    # variables with self prefix become part of the object, 
        self._number = number
        self._wins = wins
        self._losses = losses

    # a name getter method, extracts name from object
    @property # getters let you get the values of the attirbutes in the object
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter # setter let you change the values of the attirbutes in the object
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def number(self):
        return self._number
    
    # a setter function, allows uid to be updated after initial object creation
    @number.setter
    def number(self, number):
        self._number = number
    
    @property
    def wins(self):
        return self._wins

    @wins.setter
    def wins(self, wins):
        self._wins = wins 
    
    @property
    def losses(self):
        return self._losses

    @losses.setter
    def losses(self, losses):
        self._losses = losses


    
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self): # Create lets you make a new data point
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self): # lets you retrieve data in the set
        return {
            "id": self.id,
            "name": self.name,
            "number": self.number,
            "wins": self.wins,
            "losses": self.losses,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, dictionary): # lets you update and change the previous data
        for x in dictionary:
            if x == "number":
                self.number = dictionary[x]
            if x == "wins":
                self.wins = dictionary[x]
            if x == "losses":
                self.losses = dictionary[x]
        db.session.merge(self)
        db.session.commit()
        return None

    # CRUD delete: remove self
    # None
    def delete(self): # lets you delete the data or all the data
        db.session.delete(self)
        db.session.commit()
        return None

# makes the class for the user and all of the setters and getters and the crud functions
    

## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [5]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        p1 = Player(name='Patrick Mahomes', number='15', wins='64', losses='16')
        p2 = Player(name='JJ Watt', number='99', wins='77', losses='74')
        p3 = Player(name='Russell Wilson', number='3', wins='108', losses='64')
        p4 = Player(name='Travis Kelce', number='87', wins='105', losses='39')
        p5 = Player(name='Joe Burrow', number='9', wins='24', losses='17')
        p6 = Player(name='Trevor Lawrence', number='16', wins='12', losses='22')


        players = [p1, p2, p3, p4, p5, p6]

        """Builds sample user/note(s) data"""
        for x in players:
            try:
                '''add user to table'''
                object = x.create()
                print(f"Created new uid {object.name}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {x.name}, or error.")
                
initUsers()


# adds some tester data to the sqlite.db file

Created new uid Patrick Mahomes
Created new uid JJ Watt
Created new uid Russell Wilson
Created new uid Travis Kelce
Created new uid Joe Burrow
Created new uid Trevor Lawrence


## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [6]:
def find_by_name(name):
    with app.app_context():
        player = Player.query.filter_by(_name=name).first()
    return player # returns user object

# # Check credentials by finding user and verify password
# def check_credentials(uid, password):
#     # query email and return user record
#     user = find_by_uid(uid)
#     if user == None:
#         return False
#     if (user.is_password(password)):
#         return True
#     return False
        
# #check_credentials("indi", "123qwerty")

# lets you ind the user by a uid and check if the uid and password correspond with each other

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [7]:
def create():
    # optimize user time to see if uid exists
    name = input("Enter your name:")
    player = find_by_name(name)
    try:
        print("Found\n", name.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    number = input("Enter your number:")
    wins = input("Enter your wins:")
    losses = input("Enter your losses:")
    
    # Initialize User object before date
    player = Player(name=name, 
                number=number, 
                wins=wins,
                losses=losses
                )
    
    # write object to database
    with app.app_context():
        try:
            object = player.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error name {name}")
        
create()

Created
 {'id': 13, 'name': 'Kalani', 'number': 5, 'wins': 10, 'losses': 20}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [8]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = Player.query.all()
    json_ready = [player.read() for player in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1, 'name': 'Patrick Mahomes', 'number': 15, 'wins': 64, 'losses': 16},
 {'id': 2, 'name': 'JJ Watt', 'number': 99, 'wins': 77, 'losses': 74},
 {'id': 3, 'name': 'Russell Wilson', 'number': 3, 'wins': 108, 'losses': 64},
 {'id': 4, 'name': 'Travis Kelce', 'number': 87, 'wins': 105, 'losses': 39},
 {'id': 5, 'name': 'Joe Burrow', 'number': 9, 'wins': 24, 'losses': 17},
 {'id': 6, 'name': 'Trevor Lawrence', 'number': 16, 'wins': 12, 'losses': 22},
 {'id': 7, 'name': 'Patrick Mahomes', 'number': 15, 'wins': 64, 'losses': 16},
 {'id': 8, 'name': 'JJ Watt', 'number': 99, 'wins': 77, 'losses': 74},
 {'id': 9, 'name': 'Russell Wilson', 'number': 3, 'wins': 108, 'losses': 64},
 {'id': 10, 'name': 'Travis Kelce', 'number': 87, 'wins': 105, 'losses': 39},
 {'id': 11, 'name': 'Joe Burrow', 'number': 9, 'wins': 24, 'losses': 17},
 {'id': 12, 'name': 'Trevor Lawrence', 'number': 16, 'wins': 12, 'losses': 22},
 {'id': 13, 'name': 'Kalani', 'number': 5, 'wins': 10, 'losses': 20}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.

## Updating users table in sqlite.db

In [9]:
def put():
    name = str(input("Who do you want to edit?"))
    number = int(input("Enter the player's new number or same number"))
    wins = int(input("Enter the player's new number of wins"))
    losses = int(input("Enter the player's new number of losses"))
    body = {
        "name": name,
        "data": {"number": number, "wins": wins, "losses": losses}
    }
    data = body.get('data')
    player = find_by_name(name)
    with app.app_context():
        player.update(data)
        db.session.commit()
    return f"{player.name} at id {player.id} has been updated"

put()

'Kalani at id 13 has been updated'

## Deleting users table in sqlite.db

In [10]:
def delete():
    name = str(input("Who do you want to delete?"))
    player = find_by_name(name)
    with app.app_context():
        player.delete()
    return f"{player.name} at id {player.id} has been deleted"

delete()

'Kalani at id 13 has been deleted'